Type-checking is a procedure where we determine whether a given µDhall expression has a well-defined type. If so, we will have "inferred" that type and found that the expression is "well-typed". Otherwise, the expression will be considered as "ill-typed" (having a type error), and we will reject that expression as invalid and report the error to the user.

Type-checking is a necessary step before evaluating any expressions: only well-typed expressions will be evaluated.

In µDhall, type-checking is simple. The user needs to specify the types in all lambda functions, and expressions may not be recursive. So, any variable contained in a valid expression already has a specified type.

For example, consider this expression:

     λ(x : Natural) → x + y

When we look at the sub-expression `x + y`, we already know that `x` has type `Natural`. If `y` is undefined (a free variable), we cannot know the type of `y`, so we cannot type-check this expression. So, we stop and report a type error due to an undefined variable `y`. If `y` has been bound in some lambda in an outer scope (such as `λ(y : T) → ...`) then we will know the type `T` that has been assigned to `y`. After that, we need to verify that the operation `+` is well-typed: both its operands must have type `Natural`. In this case, we need to verify that `T` is `Natural`. If this is so, the entire expression is well-typed: it is a function of type `∀(x : Natural) → Natural`. Otherwise, we will have found a type error: the operation `+` is applied to a value of type different from `Natural`. We reject the expression and show the type error to the user.

We see that type-checking requires us to know what types have been previously assigned to bound variables when we descend recursively into sub-expressions. This information is held as a list of type assignments such as `[ y : T, x : Natural ]`. This list is called a **typing context** and is often denoted by the letter Γ ("gamma").

To describe this property of µDhall's type-checking algorithm, we say it is **unidirectional**. The intuition is that the type information goes from lambda function declarations (`λ(x : Natural) → ...`) towards the function bodies, but not in the opposite direction.

The unidirectional type-checking algorithm descends recursively through the expressions and their sub-expressions. Each time we encounter a lambda, such as `λ(x : Natural) → ...`, we add the type annotation `x : Natural` to the current typing context and proceed to type-check the function body using the new typing context. Checking the function body will yield success or failure, but no additional type information about `x`.

A unidirectional type-checking algorithm is not able to infer the type of `x` in the following example:

     λ(x) → λ(y : Natural) → x + y  -- Not valid in µDhall!

The binary operation `+` is defined only for `Natural` operands, but the algorithm is not able to infer that `x` must be `Natural` and to fill in the missing type annotation for `x`. That can be done via **bidirectional type-checking** algorithms, but they are much more complicated.

We also see that type-checking needs evaluation and equality comparison. In the example just shown, we need to know whether the type `T` equals `Natural`. However, `T` could be a complicated type expression involving function applications. To know whether `T` is really equal to `Natural`, we need to be able to evaluate arbitrary expressions to an unambiguous normal form. So, implementing type-checking requires us to use full alpha- and beta-normalization.

It is not a problem that beta-normalization needs type-checking to be performed first, while type-checking depends on beta-normalization. The two functions (beta-normalization and type-checking) are mutually recursive, and recursion will terminate because each subsequent recursive call will be always applied to a smaller sub-expression.

A type-checking rule is written as a judgment of the form $\Gamma \vdash t : T$. Here $\Gamma$ is a typing context (a list of type-annotated variable names), $t$ is an expression, and $T$ is a type expression. This judgment holds when we can prove that the expression $t$ has type $T$. If we cannot prove that, we say "it is a type error".

What is the difference between value expressions and type expressions? In µDhall, type expressions are written much in the same way as value expressions, except that one needs to use type symbols (such as `Natural` or `Type`), and one can use function types such as $\forall(x: T) \to ...$. Formally, type expressions are seen as expressions "of type `Type`" or "of type `Kind`". Value expressions have types such as `Natural` or `Natural → Natural`, such that the type of that type is `Type`. In this way, µDhall can formally and unambiguously distinguish between types and values. Once we implement type-checking, it will be a type error to write a µDhall program where types are used where values are required, or vice versa.

A type-checking rule is interpreted as a function from $\Gamma$ and $t$ to $T$. We will need to pattern-match on both $\Gamma$ and $t$. Pattern-matching on $\Gamma$ will be done by checking the _last_ element of the list, if it exists.

Part of type-checking is verifying that function input and output types are valid. For example, `λ(a : 123) → a` is not valid because `123` is not a type expression. Also, `λ(a : λ(T : Type) → Type) → a` is not valid because there cannot be values of type `λ(T : Type) → Type`.

We express this via a "valid function input" (VFI) judgment:

```
VFI(Type)
VFI(Kind)
```
The `VF(x)` judgment does not hold for any other `x`.

Formally, we require that `λ(a : A) → b` should have a typing `A : t` such that `VFI(t)` holds, and that `b` should have a type (say, `b : B`) such that `B` itself has a type, say, `B : u`.


Here are the rules for type-checking in µDhall:

$$
\begin{align}
& \frac{
  \begin{aligned}
    & \Gamma \vdash T : k
  \end{aligned}
}{
  \Gamma, x : T \vdash x\texttt{@0} : T
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash x\texttt{@}n : T
  \end{aligned}
}{
  \Gamma, x : A \vdash x\texttt{@}(1 + n) : T
}
; 0 < n
\\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash x\texttt{@}n : T
  \end{aligned}
}{
  \Gamma, y : A \vdash x\texttt{@}n : T
}
; x \neq y \\
& \\
& 
\frac{
}{
  \Gamma \vdash \texttt{Natural} : \texttt{Type}
} \\
& \\
& 
\frac{
}{
  \Gamma \vdash \texttt{Type} : \texttt{Kind}
} \\
& \\
& \frac{
}{
  \Gamma \vdash n : \texttt{Natural}
} \quad\text{; when }n\text{ is a }\texttt{NaturalLiteral}\\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash x : \texttt{Natural} \qquad \Gamma \vdash y : \texttt{Natural}
  \end{aligned}
}{
  \Gamma \vdash x + y : \texttt{Natural}
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash x : \texttt{Natural} \qquad \Gamma \vdash y : \texttt{Natural}
  \end{aligned}
}{
  \Gamma \vdash x * y : \texttt{Natural}
} \\
& \\
& \frac{
}{
  \Gamma \vdash \texttt{Natural/fold} : \texttt{Natural} \to \forall(N : \texttt{Type}) \to \forall(\texttt{succ} : N \to N) \to \forall(\texttt{zero} : N) \to N
} \\
& \\
& \frac{
}{
  \Gamma \vdash \texttt{Natural/subtract} : \texttt{Natural} \to \texttt{Natural} \to \texttt{Natural}
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma_0 \vdash A : i \\
    & \texttt{shift}(1, x, 0, [\Gamma_0, x : A]) = \Gamma_1 \\
    & \Gamma_1 \vdash B : o \\
    & \texttt{VFI}(i)
  \end{aligned}
}{
  \Gamma_0 \vdash \forall(x : A) \to B : o
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma_0 \vdash A_0 : c_0 \\
    & A_0 \to_\beta\, A_1 \\
    & \texttt{shift}(1, x, 0, [\Gamma_0, x : A_1]) = \Gamma_1 \\
    & \Gamma_1 \vdash b : B \\
    & \Gamma_0 \vdash \forall(x : A_1) \to B : c_1
  \end{aligned}
}{
  \Gamma_0 \vdash \lambda(x : A_0) \to b : \forall(x : A_1) \to B
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash f : \forall(x : A_0) \to B_0 \\
    & \Gamma \vdash a_0 : A_1 \\
    & A_0 \equiv A_1 \\
    & \texttt{shift}(1, x, 0, a_0) = a_1 \\
    & \texttt{sub}(x \to a_1, B_0) = B_1 \\
    & \texttt{shift}(-1, x, 0, B_1) = B_2 \\
    & B_2 \to_\beta\, B_3
  \end{aligned}
}{
  \Gamma \vdash f a_0 : B_3
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash a_0 : A \\
    & a_0 \to_\beta\, a_1 \\
    & \texttt{shift}(1, x, 0, a_1) = a_2 \\
    & \texttt{sub}(x \to a_2, b_0) = b_1 \\
    & \texttt{shift}(-1, x, 0, b_1) = b_2 \\
    & \Gamma \vdash b_2 : B
  \end{aligned}
}{
  \Gamma \vdash \texttt{let } x = a_0 \texttt{ in } b_0 : B
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash T_0 : i \qquad \Gamma \vdash t : T_1 \qquad T_0 \equiv T_1
  \end{aligned}
}{
  \Gamma \vdash (t : T_0) : T_1
} \\
& \\
& \frac{
  \begin{aligned}
    & \Gamma \vdash t : \texttt{Kind}
  \end{aligned}
}{
  \Gamma \vdash (t : \texttt{Kind}) : \texttt{Kind}
} 
\end{align}
$$

In [1]:
final case class TypeError(gamma: List[(String, Expr)], focus: Expr, target: Expr, message: String) {
    override def toString: String = s"In typing context Γ=${gamma.map{ case (v, t) => v + " : " + t.print }.mkString("[", ", ", "]")}, while type-checking ${focus.print}, type error with ${target.print}: $message"
}

def VFI(e: Expr): Boolean = e match {
    case Builtin(Type) | Builtin(Kind) => true
    case _ => false
}


def typeCheck(gamma: List[(String, Expr)], e: Expr): Either[TypeError, Expr] = {
  import Expr._

  def requireEqual(a: Expr, b: Expr): Either[TypeError, Unit] =
    if (equiv(a, b)) Right ()
    else Left(TypeError(gammal, e, a, s"Expression ${a.print} must equal ${b.print}"))

  e match {
    case Variable(name, index) => gamma match {
        case Nil => Left(TypeError(gamma, e, e, s"Variable $name not in typing context"))
        case (v, t) :: tail =>
            if (name != v) typeCheck(tail, e)
            else if (index == 0) Right(t)
            else typeCheck(tail, Variable(name, index - 1)
         }

  case Builtin(Type) => Right(Kind.!)
  case Builtin(Natural) => Right(Type.!)
  case NaturalLiteral(_) => Right(Natural.!)

  case Builtin(NaturalFold) => Right(Natural.! :~> Forall("N", Type.!, ("N".! :~> "N".!) :~> "N".! ~> "N".!))
  case Builtin(NaturalSubtract) => Right(Natural.! :~> Natural.! :~> Natural.!)

  case BinaryOp(l, op, r) if op == Operator.Plus || op == Operator.Times => for {
    t1 <- typeCheck(gamma, l)
    _  <- requireEqual(t1, Natural.!)
    t2 <- typeCheck(gamma, r)
    _  <- requireEqual(t1, Natural.!)
  } yield Natural.!

  case Annotated(body, tipe) =>
        if (betaNormalize(tipe) == Kind.! && typeCheck())
        else for {
          _ <- typeCheck(gamma, tipe)
          t1 <- typeCheck(gamma, body)
          _ <- requireEqual(t1, tipe)  
        } yield t1

        case 
}
}

SyntaxError: invalid syntax (3558879286.py, line 1)